# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_csv_file = "../output_data/Cities_Data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# load gmaps with API key and drop na/s
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

#plot Heatmap with Humidity 
humidity_map = gmaps.figure()

# Add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=200,
                               point_radius = 5)
humidity_map.add_layer(heat_layer)
#Display figure
humidity_map

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Filter any rows that don't contain the three conditions:

# A max temperature 70<x<80
ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)]

# Wind speed <10 mph
ideal_weather_df = weather_df.loc[weather_df["Wind Speed"] < 10]

# Zero cloudiness
ideal_weather_df = weather_df.loc[weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# set parameters 
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# set up the loop w/iterrow 
for index, row in hotel_df.iterrows():
    # get Lat and Lng
    lat = row["Lat"]
    lng = row["Lng"]

    # set the location with lat and lng
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # set url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing lodging info for location. Skipping...")
        pass
    
hotel_df 

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_map.add_layer(marker_layer)

humidity_map
